# Test and Benchmark

In [1]:
versioninfo()

Julia Version 1.0.3
Commit 099e826241 (2018-12-18 01:34 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin14.5.0)
  CPU: Intel(R) Core(TM) i7-3740QM CPU @ 2.70GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-6.0.0 (ORCJIT, ivybridge)


In [2]:
using Revise
using BenchmarkTools
using MendelImpute
using Random
using LinearAlgebra

┌ Info: Recompiling stale cache file /Users/biona001/.julia/compiled/v1.0/MendelImpute/DVXpm.ji for MendelImpute [e47305d1-6a61-5370-bc5d-77554d143183]
└ @ Base loading.jl:1190


## Haplotyping on complete genotype data

In [3]:
Random.seed!(123)
n = 5000 # number of individuals
p = 500  # number of SNPs
d = 500  # number of reference haplotypes
H = convert(Matrix{Float32}, rand(0:1, p, d))
X = convert(Matrix{Float32}, rand(0:2, p, n))
M = Transpose(H) * H
for j in 1:d, i in 1:(j - 1) # off-diagonal
    M[i, j] = 2M[i, j] + M[i, i] + M[j, j]
end
for j in 1:d # diagonal
    M[j, j] *= 4
end
N = Transpose(X) * H
for I in eachindex(N)
    N[I] *= 2
end
happair  = zeros(Int, n), zeros(Int, n)
hapscore = zeros(eltype(N), n)
@time haplopair!(happair, hapscore, M, N)

  0.623177 seconds (51.83 k allocations: 2.641 MiB)


In [4]:
happair

([379, 363, 75, 341, 203, 157, 31, 301, 109, 291  …  175, 27, 117, 54, 408, 235, 67, 2, 241, 167], [466, 415, 358, 447, 288, 420, 402, 388, 230, 346  …  346, 132, 410, 389, 430, 475, 363, 288, 475, 175])

In [5]:
hapscore

5000-element Array{Float32,1}:
 -374.0
 -324.0
 -334.0
 -341.0
 -298.0
 -381.0
 -393.0
 -366.0
 -357.0
 -353.0
 -348.0
 -364.0
 -349.0
    ⋮  
 -336.0
 -408.0
 -403.0
 -340.0
 -373.0
 -472.0
 -335.0
 -424.0
 -381.0
 -414.0
 -347.0
 -325.0

In [7]:
haplopair(X, H)

(([379, 363, 75, 341, 203, 157, 31, 301, 109, 291  …  175, 27, 117, 54, 408, 235, 67, 2, 241, 167], [466, 415, 358, 447, 288, 420, 402, 388, 230, 346  …  346, 132, 410, 389, 430, 475, 363, 288, 475, 175]), Float32[477.0, 446.0, 480.0, 461.0, 467.0, 469.0, 477.0, 468.0, 471.0, 454.0  …  472.0, 463.0, 470.0, 478.0, 439.0, 475.0, 427.0, 457.0, 478.0, 470.0])

In [8]:
@benchmark haplopair!(happair, hapscore, M, N)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     584.445 ms (0.00% GC)
  median time:      597.483 ms (0.00% GC)
  mean time:        599.308 ms (0.00% GC)
  maximum time:     612.100 ms (0.00% GC)
  --------------
  samples:          9
  evals/sample:     1

In [11]:
using Profile
Profile.clear()
@profile haplopair!(happair, hapscore, M, N)
Profile.print(format=:flat)

 Count File                        Line Function                               
   476 ./In[11]                       3 top-level scope                        
    21 ./array.jl                   769 setindex!                              
   476 ./boot.jl                    319 eval                                   
     1 ./broadcast.jl               555 _broadcast_getindex(::Base.Broadcast...
     1 ./broadcast.jl               582 _broadcast_getindex                    
     1 ./broadcast.jl               582 _broadcast_getindex_evalf(::typeof(S...
     1 ./broadcast.jl               794 copy                                   
     1 ./broadcast.jl               794 copy(::Base.Broadcast.Broadcasted{Ba...
     2 ./broadcast.jl               931 copyto_nonleaf!(::Array{LineNumberNo...
     2 ./broadcast.jl               515 getindex                               
     2 ./broadcast.jl               756 materialize(::Base.Broadcast.Broadca...
   477 ./essentials.jl              697 

In [12]:
@code_warntype haplopair!(X, H, M, N, happair, hapscore)

Body::Nothing
│╻           size98  1 ── %1   = (Base.arraysize)(X, 1)::Int64
││              │    %2   = (Base.arraysize)(X, 2)::Int64
││              │    %3   = (Base.arraysize)(H, 2)::Int64
│╻           mul!101 │    %4   = (H === H)::Bool
││              └───        goto #3 if not %4
││              2 ──        invoke LinearAlgebra.syrk_wrapper!(_4::Array{Float32,2}, 'T'::Char, _3::Array{Float32,2})
││              └───        goto #4
││              3 ──        invoke LinearAlgebra.gemm_wrapper!(_4::Array{Float32,2}, 'T'::Char, 'N'::Char, _3::Array{Float32,2}, _3::Array{Float32,2})
││              └───        goto #4
│╻╷╷╷╷       Colon102 4 ┄─ %10  = (Base.sle_int)(1, %3)::Bool
││╻           Type    │           (Base.sub_int)(%3, 1)
│││┃           unitrange_last    │    %12  = (Base.ifelse)(%10, %3, 0)::Int64
││╻╷╷         isempty    │    %13  = (Base.slt_int)(%12, 1)::Bool
││              └───        goto #6 if not %13
││              5 ──        goto #7
││              6 ──      

In [13]:
@time haplopair!(X, H, M, N, happair, hapscore)

  0.683807 seconds (8 allocations: 384 bytes)


In [14]:
@benchmark haplopair!(X, H, M, N, happair, hapscore)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     628.908 ms (0.00% GC)
  median time:      650.782 ms (0.00% GC)
  mean time:        651.946 ms (0.00% GC)
  maximum time:     684.798 ms (0.00% GC)
  --------------
  samples:          8
  evals/sample:     1

In [15]:
Profile.clear()
@profile haplopair!(X, H, M, N, happair, hapscore)
Profile.print(format=:flat)

 Count File                        Line Function                               
   485 ./In[15]                       2 top-level scope                        
     2 ./array.jl                   732 getindex                               
    25 ./array.jl                   769 setindex!                              
   486 ./boot.jl                    319 eval                                   
   486 ./essentials.jl              697 #invokelatest#1                        
   486 ./essentials.jl              696 invokelatest                           
     1 ./float.jl                   394 +                                      
   159 ./float.jl                   396 -                                      
   155 ./float.jl                   451 <                                      
     1 ./float.jl                   398 macro expansion                        
   342 ./simdloop.jl                 73 macro expansion                        
   486 ./task.jl                    259 

## Haplotyping on incomplete genotype data



In [23]:
Random.seed!(123)
n, p, d = 5000, 500, 500
H = convert(Matrix{Float32}, rand(0:1, p, d))
X = convert(Matrix{Float32}, rand(0:2, p, n))
M = zeros(eltype(H), d, d)
N = zeros(promote_type(eltype(H), eltype(X)), n, d)
happair  = zeros(Int, n), zeros(Int, n)
hapscore = zeros(eltype(N), n)
missingprop = 0.1

X2 = Matrix{Union{Missing, eltype(X)}}(X)
X3 = ifelse.(rand(eltype(X), p, n) .< missingprop, missing, X2)
X3_original = deepcopy(X3)
@time haploimpute!(X3, H, M, N, happair, hapscore)

  4.472586 seconds (4 allocations: 160 bytes)


In [12]:
@benchmark haploimpute!(X3, H, M, N, happair, hapscore)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     4.384 s (0.00% GC)
  median time:      4.413 s (0.00% GC)
  mean time:        4.413 s (0.00% GC)
  maximum time:     4.442 s (0.00% GC)
  --------------
  samples:          2
  evals/sample:     1

In [14]:
Profile.clear()
@profile haploimpute!(Xm, H, M, N, happair, hapscore)
Profile.print(format=:flat)

 Count File                        Line Function                               
   538 ./<missing>                   -1 anonymous                              
     1 ./linalg/matmul.jl           191 A_mul_Bt!                              
     1 ./linalg/matmul.jl           239 copytri!(::Array{Float32,2}, ::Char... 
     1 ./linalg/matmul.jl           293 syrk_wrapper!(::Array{Float32,2}, :... 
   538 ./loading.jl                 515 include_string(::String, ::String)     
   538 ./profile.jl                  23 macro expansion                        
     1 ./simdloop.jl                 73 macro expansion                        
   538 ./task.jl                    335 (::IJulia.##11#14)()                   
   538 ...IJulia/src/eventloop.jl     8 eventloop(::ZMQ.Socket)                
   538 .../src/execute_request.jl   160 execute_request(::ZMQ.Socket, ::IJu... 
     3 ...pute/src/haplotyping.jl   148 fillmissing!(::NullableArrays.Nulla... 
     3 ...pute/src/haplotyping.jl   149 